In [1]:
:opt no-lint
:opt show-types

# Functor, Applicative, Monad
https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/

기본적으로 위 내용을 노트북으로 옮기면서 번역을 나름대로 군데군데 제가 말하기 편하게 고치기도 했습니다.

----

여기 단순한 값(value)이 있어.

![value](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/value.png)

In [2]:
v :: Int
v = 2

v :: Int

In [3]:
v

2

그리고 이 값에 함수(function)를 적용하는(apply) 법을 알지.

![value apply](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/value_apply.png)

In [4]:
f3 :: Int -> Int
f3 = (+3)

f3 :: Int -> Int

In [5]:
f3 v

5

In [6]:
(+3) 2

5

뭐 이건 아주 간단하지.

그럼 이제 이 값이 어떠한 틀(context)안에 들어있다고 하자.

값을 담을 수 있는 이런 틀을 상자로 생각해 볼 수 있어.

![value and context](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/value_and_context.png)

In [7]:
cv = Just v

cv :: Maybe Int

In [8]:
cv

Just 2

함수를 이런 틀에 포장된 값에 적용하면 **어떤 틀이냐에 따라** 다른 결과값이 되겠지.

바로 이런 아이디어로부터 Functor, Applicative, Monad라는 것이 나왔어.

`Maybe` 데이타 타입은 다음과 같은 두 종류의 틀을 정의하고 있어.

![maybe](http://adit.io/imgs/functors/context.png)

```haskell
data Maybe a = Nothing | Just a
```

이제 함수를 `Just a`에 적용하는지 `Nothing`에 적용하는지에 따라 어떻게 달라지는지 곧 보게 될거야.

우선은 Functor에 대해 알아보도록 하자고.

----
## Functor
상자에 담긴 값에는 앞에서 쓰던 보통의 함수를 적용할 수가 없어.

![no fmap ouch](http://adit.io/imgs/functors/no_fmap_ouch.png)

In [9]:
:type f3  -- (+3)
:type cv -- Just 2

f3 :: Int -> Int

cv :: Maybe Int

In [10]:
f3 cv -- (+3) (Just 2)

: 

여기서부터 `fmap` 등장할 시간이야.
이 동네에 출신인 `fmap`은 이 상자에 빠삭하거든.
그래서 `fmap`은 틀 안에 있는 값에 함수를 어떻게 적용해야 하는지 알고 있어.
예컨대, `(+3)`을 `Just 2`에 적용하고 싶다면, fmap으로 이렇게 해봐.

![fmap apply](http://adit.io/imgs/functors/fmap_apply.png)

**쨘!** `fmap`이 멋지게 해냈어.

그런데 `fmap`은 이렇게 함수를 적용시키는 방법을 어떻게 알고 있는 걸까?

### Functor가 도대체 뭐야?

Functor는 이렇게 정의된 타입 클래스(type class)야.
```haskell
class Functor f where
  fmap :: (a -> b) -> (f a -> f b)
```

  1. `f`라는 데이타 타입(좀더 정확히는, 데이타 타입 생성자)이 `Functor`가 되려면
  1. 단, `fmap`이 각각의 데이타 타입마다 어떻게 동작할지 정의해줘야 해

![functor def](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/functor_def.png)

`Functor`란 `fmap`을 적용시키는 방법을 정의하여 제공해야 하는 데이터 타입 생성자(`f`)야.

`fmap`은 이런 식으로 동작하도록 정의해줘야 하지.

![fmap def](http://adit.io/imgs/functors/fmap_def.png)
  1. `fmap`은 `a -> b` 타입의 함수를 받아서 (예: `even`)
  1. `f a`, 즉 `a`에 대한 `Functor`를 받아 (예: `Just 2`)
  1. 결과로 `f b`, 즉 `b`에 대한 `Functor`를 돌려줌 (예: `Just True`)
 
한번 실제로 해보자고.

In [11]:
:type f3  -- (+3)
:type cv -- Just 2
:type fmap f3

f3 :: Int -> Int

cv :: Maybe Int

fmap f3 :: forall (f :: * -> *). Functor f => f Int -> f Int

In [12]:
fmap f3 cv
fmap f3 Nothing

Just 5

Nothing

In [13]:
:type (even :: Int -> Bool)
:type cv
:type fmap (even :: Int -> Bool)

(even :: Int -> Bool) :: Int -> Bool

cv :: Maybe Int

fmap (even :: Int -> Bool) :: forall (f :: * -> *). Functor f => f Int -> f Bool

In [14]:
fmap even cv

Just True

마치 마법처럼 `fmap`이 함수들을 적용할 수 있었던 건 `Maybe`가 Functor이기 때문이야.

그건 바로 이렇게 `fmap`이 `Just`와 `Nothing`에 대해
구체적으로 어떻게 동작해야 하는지 명세하고 있다는 뜻이지.

```haskell
instance Functor Maybe where
  fmap func (Just val) = Just (func val)
  fmap func Nothing    = Nothing
```

여기, `fmap (+3) (Just 2)`라고 쳤을때 무대 뒤에서는 어떠한 일이 일어나는지 봐봐.

![fmap just](http://adit.io/imgs/functors/fmap_just.png)

그래 그렇다면 `fmap` 이번에는 `(+3)`을 `Nothing`에다 적용해보자고.

![fmap nothing](http://adit.io/imgs/functors/fmap_nothing.png)

In [15]:
fmap (+3) Nothing
fmap even Nothing

Nothing

Nothing

![bill](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/bill.png) <br> Maybe functor도 모르는 Bill O’Reilly

메트릭스의 모피어스처럼 `fmap`은 무엇을 해야할지 알고 있지. `Nothing`으로 시작하면, `Nothing`으로 끝난다! `fmap` 참 끝내주지.

이번엔 `Maybe` 데이터 타입을 왜 만들었는지 알아보자.
`Maybe`같은 걸 잘 사용하지 않는 언어에서
예를 들어 데이터를 가지고 작업을 한다고 한다면 보통 이런 식으로 할거야.
```ruby
post = Post.find_by_id(1)
if post
  return post.title
else
  return nil
end
```

하지만 하스켈에서라면
```haskell
fmap (getPostTitle) (findPost 1)
```

만일 `findPost`가 게시글(post)를 찾아서 돌려준다면 `getPostTitle`로 제목(title)을
얻을 수 있겠지. 한편 `Nothing`을 돌려준하다면 `Nothing`을 얻을 것고. 어때, 깔끔하지? 

`fmap`의 중위표기(infix) 버전인 `<$>`를 써서 이렇도 종종 이렇게 쓰기도 해.

```haskell
getPostTitle `fmap` (findPost 1)
getPostTitle <$> (findPost 1)
```

### 리스트도 Functor라니!
다른 예를 하나 더 살펴보자. 리스트에 (원소 각각에 대한) 함수를 적용하려면 어떻게 해야 할까?

![fmap list](http://adit.io/imgs/functors/fmap_list.png)

리스트도 Functor라는 사실! 정의는 이렇지.
```haskell
instance Functor [] where
   fmap = map
```

### 아니 함수마저 Functor라고?!

자자, 이제 마지막으로 예를 들어 이런 식으로 함수에 다른 함수를 적용하려면 어떻게 해야 할까?
```haskell
fmap (+3) (+1)
```
![이뭔개](https://kin-phinf.pstatic.net/20190522_215/1558522956633VSrs6_GIF/%EC%9E%A5%ED%95%AD%EC%84%A0%ED%9D%91%EA%B0%9C.gif)

처음 보면 어이가 없겠지만 어떻게 되나 한번 찬찬히 살펴보자고.

함수 하나를 이렇게 그림으로 생각해 보자고.

![function with a value](http://adit.io/imgs/functors/function_with_value.png)

여기 함수를 다른 함수에 (`fmap`을 통해) 적용하는 것을 이렇게 생각할 수 있어.

![fmap function](http://adit.io/imgs/functors/fmap_function.png)

그러니까 함수도 Functor야!
```haskell
instance Functor ((->) r) where
  fmap func1 func2 = func1 . func2
```

  * `fmap :: (a -> b) -> ((->) r a) -> ((->) r b)` <br>
    즉 `fmap :: (a -> b) -> (r -> a) -> (r -> b)`
  * `func1 :: a -> b`
  * `func2 :: (->) r a` 즉 `func2 :: r -> a`

In [16]:
:type fmap

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

그러니까 함수에 대한 `fmap`은 좀 싱겁지만 그냥 두 함수의 합성일 뿐!

In [16]:
foo = fmap (+3) (+2)
foo 10

foo :: forall b. Num b => b -> b

15

----
### 연습문제
다음 `Tree`에 대한 `Functor` 인스턴스를 정의해 보라.

예컨대
```haskell
fmap even (Node 2 (Node 1 Null Null)
                  (Node 3 Null Null) )
   ==  (Node True (Node False Null Null)
                  (Node False Null Null) ) 
```

In [18]:
data Tree a = Null
            | Node a (Tree a) (Tree a)
            deriving (Eq,Show)

In [19]:
{-
instance Functor Tree where
  fmap f Null           = undefined
  fmap f (Node x t1 t2) = undefined
-}

----

## Applicative Functor
Applicative는 여기서 한 단계 더 나가는 거야.

Applicative도 값을 어떤 틀에 담을 수 있는 건 Functor와 마찬가지긴 하지만

![value and context](http://adit.io/imgs/functors/value_and_context.png)

거기서 한 단계 더 나아가 함수까지 틀에 담아 놓을 수 있다는 거지!

![function and context](http://adit.io/imgs/functors/function_and_context.png)

그럼 더 깊숙히 들여다보자고. Applicative는 만만하지 않아.

`Control.Applicative`에서는 상자 안에 함수를 다른 상자 안에 값에 적용할 줄 아는
`<*>` 연산자를 제공하고 있어.

![applicative just](http://adit.io/imgs/functors/applicative_just.png)

In [17]:
Just (+3) <*> Just 2

Just 5

In [18]:
Just even <*> Just 2

Just True

`<*>`를 사용해보면 꽤나 흥미진진하지. 이걸 봐봐.

In [19]:
[(*2), (*3)] <*> [1, 2, 3]

[2,4,6,3,6,9]

![applicative list](http://adit.io/imgs/functors/applicative_list.png)

여기 Applicative면 할 수 있지만 Functor로만은 못하는 일이 있어.

어떤 함수를 두 개의 (혹은 그 이상의) 상자 안에 들어 있는 값들에 적용하려면 어떻게 해야 할까?

In [23]:
-- (+) :: Int -> (Int -> Int)
justPlus5 = (+) <$> Just (5 :: Int)  -- Just (+5)

justPlus5 :: Maybe (Int -> Int)

방금 위에까지는 Functor로 되는거지. 근데 그 다음부턴 안되.

In [24]:
justPlus5 <$> Just (3 :: Int)
-- Just (+5) <$> Just 3

: 

Appplicative를 쓰면 이렇게

In [25]:
Just (+5) <*> Just 3

Just 8

Applicative는 Functor를 슥 밀쳐내면서 말하지.

"어른들은 여러개의 인자를 다루는 함수를 쓴다 말이야"

"난 `<$>`와 `<*>`로 둘 다로 무장하고 있어서 상자 안에 함수에 상자 안에 값을 몇개든 필요한 만큼 붙여서 계산해서 값이 들어있는 상자를 만들어 낼 수 있다! 이말이야, ㅎㅎㅎㅎ”

In [26]:
(+) <$> Just 5 <*> Just 3
(*) <$> Just 5 <*> Just 3

Just 8

Just 15

In [28]:
(+) <$> Just 5  <*> Nothing
(+) <$> Nothing <*> Just 3

Nothing

Nothing

참, 그리고 함수 하나로 개수가 2개인 경우 같은 일을 하는 `liftA2`라는 함수도 있어.

In [29]:
import Control.Applicative

liftA2 (+) (Just 5) (Just 3)
liftA2 (*) (Just 5) (Just 3)

Just 8

Just 15

----
## Monad
모나드를 배우는 방법:
 1. 컴퓨터 과학 박사학위를 딴다.
 1. 필요없으니 집어 치운다!

그러지 말고 Monad 쉽게 알아보자고.

Functor는 함수를 상자 안의 값에 적용할 수 있지.

![fmap](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/fmap.png)

Applicatives는 상자 안 함수를 상자 안의 값에 적용할 수 있지.
![applicative](http://adit.io/imgs/functors/applicative.png)

Monad는 상자로 포장된 값을 **(상자없이 그냥 보통의 값을 받아 경우에 따라 다른) 상자를 만들어내는 함수**에다가 넘길 수 있도록 해준다는 거야.
Monad에는 바로 이런 일은 하는 ("bind"라고 부르는) 연산자 `>>=`가 있어.

예를 들어 한번 보자고, 지금까지 계속 봐오던 `Maybe`가 바로 모나드야.

![context](http://adit.io/imgs/functors/context.png)
```haskell
data Maybe a = Nothing | Just a
```

짝수에 대해서만 의미있는 일을 하는 `half`라는 함수를 생각해 보자고.


In [31]:
half :: Int -> Maybe Int
half x = if even x
            then Just (x `div` 2)
            else Nothing

half :: Int -> Maybe Int

![half](http://adit.io/imgs/functors/half.png)

근데 상자로 포장된 값을 쑤셔 넣으려 하면 그냥은 잘 안되겠지?

![half](http://adit.io/imgs/functors/half_ouch.png)

그래서 값이 들어있는 상자를 쑤셔 넣으려면 `>>=`가 필요하단 말입니다.

어 ... 이게 `>>=`의 사진이래 (믿거나 말거나)

![plunger](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/plunger.jpg)

어쨌뜬 이렇게 돌아간다는 거야.

In [32]:
Just 3  >>= half
Just 4  >>= half
Nothing >>= half

Nothing

Just 2

Nothing

안에서는 어떤 일이 벌어지고 있는 걸까? `Monad`는 또 다른 타입 클래스(type class)야.
여기 그 타입 클래스 정의 중에서 일부분만 보여줄께.
```haskell
class Monad m where
    (>>=) :: m a -> (a -> m b) -> m b
```

여기서 `>>=`는 이렇게 동작하도록 (`m`마다 다르게) 정의해줘야 해.
  1. `a`값이 들어있는 `m`상자를 받고
  1. `a`값을 받아 `b`값이 든 상자를 만들어내는 함수를 받아서
  1. 결과적으로 `b`가 들어있는 `m`상자를 돌려주지
  
![bind def](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/bind_def.png)

이렇게 `Maybe`를 모나드로 정의할 수 있어
```haskell
instance Monad Maybe where
    Nothing >>= func  = Nothing
    Just v  >>= func  = Just (func v)
```

그럼 `Just 3`에 대해 어떻게 동작하는지 보자고!

![monad just](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/monad_just.png)

`Nothing`을 넣는 경우야 뭐 더 간단하지.

![monad nothing](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/monad_nothing.png)

이렇게 연달아 호출할 수도 있어.

In [34]:
Just 20 >>= half
Just 20 >>= half >>= half
Just 20 >>= half >>= half >>= half

Just 10

Just 5

Nothing

![monad chain](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/monad_chain.png)


![whoa](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/whoa.png) 우왕 끝내주는데!

이제 `Maybe`가 `Functor`이자, `Applicative`이며, `Monad`라는 거 알았지?

이제 슬슬, `IO` 모나드 예제를 볼 때가 되었군.

(주의사항: 이 `IO` 예제들 중 키보드 입력과 관련된 내용은 노트북이 아닌 Terminal에서 할 것!)

![io](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/io.png) 숫자 10아님 ... 영어 `IO`임

세개의 특별한 함수가 있습니다. `getLine`은 인자를 받지 않고, 사용자의 입력을 받습니다.
```haskell
getLine :: IO String
```

![getLine](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/getLine.png)

`readFile`은 문자열(파일명)을 받아, 파일에 있는 내용을 반환합니다.
```haskell
readFile :: FilePath -> IO String
```

![readFile](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/readFile.png)

`putStrLn`은 문자열을 받아 출력하고 줄을 바꿉니다.
```haskell
putStrLn :: String -> IO ()
```

![putStrLn](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/putStrLn.png)

여기 나온 세개의 함수는 평범한 값을 받아서 (또는 아예 값을 받지 않고),
값이 들어있는 `IO`상자를 돌려줍니다. 그래서 이런 함수를 `>>=`로 엮을 수 있지!
```haskell
getLine >>= readFile >>= putStrLn
```

![monad io](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/monad_io.png)

오 예! 모나드 쇼가 펼쳐졌습니다!

덧붙이자면, 하스켈은 이 `Monad`에 대해, `do`라는 문법설탕(syntax suger)를 제공하고 있어.
```haskell
foo = do
    filename <- getLine
    contents <- readFile filename
    putStrLn contents
```

---
## 정리하며
 1. functor란 `Functor` 타입 클래스를 구현한 데이타 타입 (생성자)
 1. applicativ란 `Applicative` 타입 클래스를 구현한 데이터 타입 (생성자)
 1. monad는 `Monad` 타입 클래스를 구현한 데이타 타입 (생성자)
 1. `Maybe`의 경우 이 세 가지를 모두 구현하므로 functor이자 applicative이며 moand이다

이 세 개념은 어떻게 다를까?

![recap](https://netpyoung.github.io/external/functors_applicatives_and_monads_in_pictures/img/recap.png)
  * functor: `fmap` 혹은 `<$>`로 상자 속 값에 함수 적용 가능
  * applicative: `liftA`이나 `<*>`로 상자 속 함수를 상자 속 값에 적용 가능
  * monad: `liftM`나 `>>=`로 값이 들어있는 상자를 보통의 값을 받아 상자를 만들어내는 함수에 쑤셔넣을 수 있음

친구들아 (여기까지 왔으면 친구 맞겠지), 이제는 모나드가 쉬우면서도 스마트한 아이다어라는 것에 공감할 거라 생각해. 하지만 이 가이드는 시작일 뿐이고 더 많은 것들이 있어. LYAH에서 [Monad에 대해 정리한 장](http://learnyouahaskell.com/a-fistful-of-monads)도 한번 살펴보길 바라. Miran이 워낙 깊이 잘 설명을 해놔서 여기선 많은 것을 얼버무리고 넘어갔어.

----
이 글이 번역된 영문 블로그 글은 여기에

http://adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html

더 많은 모나드와 그림을 원하신다면, [three useful monads](http://adit.io/posts/2013-06-10-three-useful-monads.html)를 확인하시기 바랍니다.

---